In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv("Churn_Modelling.csv")

In [3]:
X = dataset.iloc[:,3:13]
y = dataset.iloc[:,13]

In [4]:
# Create dummy variable 
geography  = pd.get_dummies(X["Geography"], drop_first = True)
gender = pd.get_dummies(X["Gender"], drop_first = True)

In [5]:
X = pd.concat([X,geography,gender], axis = 1)

In [6]:
X= X.drop(["Geography","Gender"], axis =1)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid

In [20]:
def create_model(layers, activation):
    model = Sequential()

    for i, nodes in enumerate(layers):
        if i == 0:
            model.add(Dense(nodes, activation=activation, input_dim=X_train.shape[1]))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes, activation=activation))
            model.add(Dropout(0.3))

    model.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [22]:
model = KerasClassifier(model=create_model, verbose=0)

In [23]:
layers = [[20], [40,20], [45,30,15]]
activations = ['sigmoid', 'relu']

param_grid = {
    "model__layers": layers,
    "model__activation": activations,
    "batch_size": [128, 256],
    "epochs": [30]
}

In [24]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)

In [25]:
grid_result = grid.fit(X_train, y_train)

C:\Users\Gaurav\Desktop\ML Projects\KrishNaik\env\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Gaurav\Desktop\ML Projects\KrishNaik\env\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Gaurav\Desktop\ML Projects\KrishNaik\env\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model inst

C:\Users\Gaurav\Desktop\ML Projects\KrishNaik\env\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


C:\Users\Gaurav\Desktop\ML Projects\KrishNaik\env\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Gaurav\Desktop\ML Projects\KrishNaik\env\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Gaurav\Desktop\ML Projects\KrishNaik\env\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model inst

In [26]:
[grid_result.best_score_,grid_result.best_params_]

[np.float64(0.857),
 {'batch_size': 128,
  'epochs': 30,
  'model__activation': 'relu',
  'model__layers': [45, 30, 15]}]

In [27]:
pred_y = grid.predict(X_test)

In [29]:
y_pred = (pred_y > 0.5)

In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

0.8605